In [ ]:
import requests,json,datetime, types, ibm_db # need to install these libraries to run
##All TWC visual products have a product name and number
weather_package = 'Severe'
prod_num = [339, 328, 334, 336, 338, 335, 401] # Radar Derived 24-Hour HailVision. 
prod_variable_id=['MESHMax30min', 'MESHMax60min', 'MESHMax120min', 'MESHMax180min', 'MESHMax360min', 'MESHMax1440min', 'MESHTrack060min'] # Per TWC naming conventions
prod_decription = ['Radar Derived 30-Minute HailVision', 'Radar Derived 1-Hour HailVision', 'Radar Derived 2-Hour HailVision', 'Radar Derived 3-Hour HailVision', 'Radar Derived 6-Hour HailVision', 'Radar Derived 24-Hour HailVision', 'Radar Derived 1-Hour HailVision']
APIKEY='THE_WEATHER_COMPANY_WEATHER_DATA_PACKAGE_KEY_HERE'  #Enter your API key 

In [ ]:
########################PRODUCT INVENTORY CALL################################  
def InvCall(product_number, product_variable_id_string):
    base1 = 'https://api.weather.com/v2/tiler/info?products='+ str(product_number) +':'+ product_variable_id_string
    base2 = '&apiKey='+APIKEY
    url = base1 + base2 
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        for x in range(0,1):
            for y in range(0,1):
                timestamp = data["layers"][str(product_number)][product_variable_id_string]["dimensions"][x]["t"][y]
            return timestamp

In [ ]:
##########################   API CALL   ###################################
def APIcall(product_number, product_variable_id_string, product_descr, timestamp,ts): # function for pulling data 
    count=0

    num_layers = 6  #10
    inc=12.7  
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
   
    low_limit = 12.7   ##Thresholds are in mm, min is 5 mm (0.2") up to 50 mm (~2") 
    low_limitS = str(low_limit)
    high_limit=low_limit+12.7
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+ str(product_number)+':'+ product_variable_id_string
    base2 = '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'
    
    for x in range(0,num_layers):
        
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print (url)
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']
            #convert to inches
            HailSizeIn= round(float(y['properties']['threshold'])*0.0394,1)
            f['properties']['HailSize'] = HailSizeIn  # y['properties']['threshold']
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
          
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit) 
    
    with open("HailVision_"+ts+".geojson", "w") as outfile:
        json.dump(fc, outfile)            
    with open("HailVision.geojson", "w") as outfile:
        json.dump(fc, outfile) 
    
    now=datetime.datetime.utcnow()
    year=str(now.year)
    month=str('{:02d}'.format(now.month))
    day=str('{:02d}'.format(now.day))
    hour=str('{:02d}'.format(now.hour))
    ts=year +"-"+month+"-"+day+" "+hour+"00"
    day_date_time = month + '/' + day + '/' + year + " " + hour + ":00"
    ibm_db_conn = ibm_db.connect("DATABASE=BLUDB;HOSTNAME=getYourOwnHost.cloud.ibm.com;PORT=50001;PROTOCOL=TCPIP;UID=YOUR_USERID;PWD=YOUR_DB2_PASSWORD_HERE;Security=SSL;", "YOUR_USERID", "YOUR_DB2_PASSWORD_HERE")
    sql = "INSERT INTO HISTORICAL_SEVERE_WEATHER VALUES('" + weather_package + "', " + str(product_number) + ",'" + product_variable_id_string + "','" + product_descr + "','" + day_date_time + "','NA', ?);"       
    print(sql)
    stmt = ibm_db.prepare(ibm_db_conn, sql)   
    #print(fc)
    ibm_db.bind_param(stmt, 1, str(fc))
    ibm_db.execute(stmt)  
   

In [ ]:
################################################################################
i = 0
for product in prod_num:
    timestamp = InvCall(prod_num[i], prod_variable_id[i])  
    now=datetime.datetime.utcnow()
    year=str(now.year)
    month=str('{:02d}'.format(now.month))
    day=str('{:02d}'.format(now.day))
    hour=str('{:02d}'.format(now.hour))
    ts=year +"-"+month+"-"+day+" "+hour+"00"
    APIcall(prod_num[i], prod_variable_id[i], prod_decription[i], timestamp,ts)
    i = i + 1